## IOCs

In [12]:
# Collecting information IOCs
blind_files = ["/etc/resolv.conf", "/etc/motd", "/etc/issue", "/etc/passwd", "/etc/shadow"]
system = ["uname -a", "ps aux", "top -n 1 -d", "uname -m", "w"]
networking = ["hostname -f", "ip addr show", "ip ro show", "ifconfig -a", "route -n"]
user_accounts = ["cat /etc/passwd", "cat /etc/shadow", "/etc/security/passwd", ]
user_info = ["ls -alh /home/*/", "ls -alh /home/*/.ssh/", "cat /home/*/.ssh/known_hosts", "cat /home/*/.ssh/authorized_keys", "cat /home/*/.hist"]
creds = ["/home/*/.ssh/id*", "tmp/krb5cc_*", "/tmp/krb5.keytab", "/home/*/.gnupg/secrings.gpgs"]
configs = ["ls -aRl /etc/ * awk '$1 ~ /w.$/' * grep -v lrw 2>/dev/nullte", "cat /etc/issue{,.net}", "cat /etc/master.passwd", "cat /etc/hosts", "cat /etc/group"]
distro = ["uname -a", "lsb_release -d", "/etc/issue", "cat /etc/*release", "cat /etc/debian_version"]
package = ["rpm -qa --last | head", "yum list | grep installed", "dpkg -l", "dpkg -l | grep -i \"linux-image\"", "dpkg --get-selections"]
package_source = ["cat /etc/apt/sources.list", "ls -l /etc/yum.repos.d", "cat /etc/yum.conf"]
important_files = ["ls -dlR */", "ls -alR | grep ^d", "find /var -type d", "ls -dl `find /var -type d`", "ls -dl `find /var -type d` | grep -v root"]



## Driver Functions

In [39]:
# Set up files to download

def user_file_download():
    """Ask the user what file they want to download and check if that file exists or not"""
    fileToGet = input("What file do you want to get from the remote machine? ")
    filename = fileToGet.split("/")
    filename = filename[-1]
    if os.path.exists("./remoteFiles/" + filename):
        while True:
            delete = input("A file with the same name already exists on your system. Would you like to delete it? (y/n)")
            if delete.lower() == "y":
                os.remove("./remoteFiles/" + filename)
                break
            elif delete.lower() == "n":
                break
    return fileToGet

def user_file_upload():
    """Get the file that the user wants to upload to the remote server"""
    while True:
        fileToPost = input("What file would you want to upload? ")
        if os.path.exists(fileToPost):
            return fileToPost
        else:
            try_again = input("That file does not exist on your system. Would you like to retry? (y/n)")
            if try_again.lower() == "y": 
                break
            else:
                exit()

def download_file(file):
    """Download the file over an already established ssh connection"""
    # Start SFTP session
    sftp = ssh.open_sftp()

    # Files to download
    filename = file.split('/')

    # Local path to store files
    local_path = "remoteFiles/" + filename[-1]

    sftp.get(fileToGet, local_path)

    ssh.close()

def upload_file(file):
    """Upload a file to an already established ssh connection"""
    # Start SFTP session
    sftp = ssh.open_sftp()

    # Files to Upload 
    filename = file.split('/')

    # Remote path to put files
    remote_path = filename[-1]

    sftp.put(fileToPost, remote_path)

    ssh.close()


## Set up SSH connection

In [51]:
import os
import paramiko
from getpass import getpass

# ask the user if they want to upload or download
uod = input("Would you like to upload a file or download a file? (u/d)")
if uod.lower() == "d":
    fileToGet = user_file_download()
elif uod.lower() == "u":
    fileToPost = user_file_upload()

#password prompt
thePass = getpass(prompt="Please enter SSH password: ")

#Host information
host = "192.168.1.12"
port = 22
username = "sam-adm"
password = thePass

#attempt the connection
try: 
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)
except paramiko.AuthenticationException:
    print("Authentication Failed")


## Download File

In [53]:
# Download the file the user requested
try:
    download_file(fileToGet)
except AttributeError:
    print("No open SSH connection")

## Upload File

In [40]:
# Upload the file the user specified
try:
    upload_file(fileToGet)
except AttributeError:
    print("No open SSH connection")

No open SSH connection


## Run Command

In [52]:
# Run our week 4 script on the remote system
try:
    ssh.exec_command("python3 fs.py -d /usr/bin > results.body")
except AttributeError:
    print("No open SSH connection")